In [ ]:
# default_exp utils

# utils

> General helper functions

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import re
import functools
import tempfile
import pkgutil
import subprocess
from configparser import ConfigParser

In [ ]:
#export
def check_git_dir():
    "Check whether there is a .git directory in your current folder"
    return bool(os.path.exists(".git"))

In [ ]:
#export
def check_project_root(func):
    """Check that the user is on the project root directory by checking if there is a .git folder
    before running `func`
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        if not check_git_dir():
            raise Exception("You need to be on your project root to execute this command")
        else:
            return func(*args, **kwargs)
    return wrapper

In [ ]:
#export
@check_project_root
def read_credentials(path):
    "Read credentials from `path`"
    config = ConfigParser(delimiters=['='])
    with open(path, "r") as f:
        config.read_string("[DEFAULT]\n" + f.read())
    return config["DEFAULT"]

In [ ]:
if not check_git_dir():
    os.chdir("..")  # we need this if we test locally in the notebook itself
input_credentials = "user = SomeUser\npassword = SomePassword"
expected_credentials = {"user": "SomeUser", "password": "SomePassword"}
with tempfile.TemporaryDirectory() as tmp_dir:
    tmp_file = os.path.join(tmp_dir, "tmp")
    with open(tmp_file, "w") as f:
        f.write(input_credentials)
    credentials = read_credentials(tmp_file)
assert credentials == expected_credentials

In [ ]:
#export
def get_config():
    "Get config from settings.ini"
    config = ConfigParser(delimiters=['='])
    config.read('settings.ini')
    cfg = config['DEFAULT']
    return cfg

In [ ]:
#export
def get_template(name):
    "Get template with filename `name`"
    template = pkgutil.get_data(__name__, f"templates/{name}").decode("utf-8")
    return template

In [ ]:
#export
@check_project_root
def update_gitignore():
    "Update .gitignore with stuff we do not want to track"
    print("Updating .gitignore")
    add_ignore_files = [
        ".gh-credentials",
        "conda-recipe/*",
        "conda-bld/*",
        
    ]
    if os.path.exists(".gitignore"):
        with open(".gitignore", "r") as f:
            ignore_files = f.read().split("\n")
        for ifile in add_ignore_files:
            if ifile not in ignore_files:
                ignore_files.append(ifile)
        ignore_file = "\n".join(ignore_files)
    else:
        ignore_file = "\n"
    with open(".gitignore", "w") as f:
        f.write(ignore_file)

In [ ]:
#export
@check_project_root
def write_settings_ini(project_type):
    "Write settings.ini from template"
    print("Generating settings.ini")
    if os.path.exists("settings.ini"):
        print("settings.ini already exists. Skipping this step")
    else:
        temp_name = "settings_template.ini" if project_type == "package" else "settings_slim_template.ini"
        template = get_template(temp_name)
        with open("settings.ini") as f:
            f.write(template)

In [ ]:
#export
@check_project_root
def write_setup_py():
    "Write setup.py from template"
    print("Generating setup.py")
    if os.path.exists("setup.py"):
        print("setup.py already exists. Skipping this step")
    else:
        template = get_template("setup_template.py")
        with open("setup.py") as f:
            f.write(template)

In [ ]:
#export
@check_project_root
def write_conda_build_scripts():
    "Write bld.bat and build.sh scripts for building conda packages"
    "Generating bld.bat and build.sh for conda recipe"
    if not os.path.exists("conda-recipe"):
        os.mkdir("conda-recipe")
    temp_map = {
        "bld.bat": "bld_template.bat",
        "build.sh": "build_template.sh"
    }
    for file_name, temp_name in temp_map.items():
        print(f"Generating file {file_name}")
        if os.path.exists(os.path.join("conda-recipe", file_name)):
            print(f"{file_name} already exists. Skipping this step")
        else:
            template = get_template(temp_name)
            with open(os.path.join("conda-recipe", file_name), "w") as f:
                f.write(template)

In [ ]:
#export
def get_conda_env_packages():
    "Get conda environment packages"
    packages = subprocess.run(["conda", "list", "--explicit"], capture_output=True).stdout.decode("utf").splitlines()
    packages = [p for p in packages if not re.match(r"^(?:#|@)", p) and not re.search("dev", p.split("=")[-1])]
    return packages

In [ ]:
#export
@check_project_root
def get_anaconda_credentials():
    "Get anaconda credentials from .anacondarc in your home directory"
    ana_file = ".anaconda-credentials"
    user_root = os.path.expanduser("~")
    if os.path.exists(ana_file):
        cfg = read_credentials(ana_file)
    elif os.path.exists(os.path.join(user_root, ana_file)):
        cfg = read_credentials(os.path.join(user_root, ana_file))
    else:
        raise Exception(
            "No .anaconda-credentials file found. " 
            "Please save your anaconda credentials "
            "either in this project's root directory or under your home directory"
        )
    return (cfg["username"], cfg["password"])

In [ ]:
#export
def write_init_version():
    "Write __version__ in __init__.py if it is a package"
    cfg = get_config()
    if "lib_name" in cfg:
        package_name = cfg["lib_name"]
        version = cfg["version"]
        if os.path.exists(package_name):
            init_path = os.path.join(package_name, "__init__.py")
            if not os.path.exists(init_path):
                print("No __init__.py file found. Generating it")
                with open(init_path, "w") as f:
                    f.write(f'__version__ = "{version}"')
            else:
                with open(init_path, "r") as f:
                    init_lines = f.readlines()
                has_version = max([True if "__version__" in line else False for line in init_lines])
                if has_version:
                    print("Updating __version__ in __ini__.py")
                    update_init_lines = [
                        re.sub(r'__version__ = ".*"', f'__version__ = "{version}"', line)
                        for line in init_lines
                    ]
                else:
                    update_init_lines += [f'\n__version__ = "version"']
                update_init = "".join(update_init_lines)
                with open(init_path, "w") as f:
                    f.write(update_init)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_gh.ipynb.
Converted 02_conda.ipynb.
Converted 03_cli.ipynb.
Converted 04_utils.ipynb.
Converted index.ipynb.
